In [ ]:
!pip install langchain openai langchain-openai google-cloud-secret-manager scipy datasets

In [ ]:


from transformers import pipeline
import scipy
from langchain import PromptTemplate, LLMChain
from langchain_openai import OpenAI
import torch
from IPython.display import Image
from datasets import load_dataset
from google.colab import userdata
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

secret_string = userdata.get('OPENAI_TOKEN')

llm = OpenAI(openai_api_key=secret_string, temperature=1)

captioner = pipeline("image-to-text",model="Salesforce/blip-image-captioning-base", max_new_tokens=20)
parser = StrOutputParser()


def image2text(image):
    text_result = captioner(image)
    return text_result[0]['generated_text']

def crear_historia(topic):
   template = """
   You are a writer and story teller.
   Your task is generate short stories from a shrot description. The story cannot have more than 50 words.
   CONTEXT: {topic}
   STORY:
   """
   topic_template = PromptTemplate(template=template, input_variables=['topic'])

   chain = ({"topic": RunnablePassthrough()} | topic_template | llm | parser)
   response = chain.invoke(topic)
   return response

def crear_audio(text):
  synthesiser = pipeline("text-to-speech", model="microsoft/speecht5_tts")
  embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
  speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

  speech = synthesiser(f"{text}", forward_params={"speaker_embeddings": speaker_embedding})

  scipy.io.wavfile.write("historia.wav", rate=speech["sampling_rate"], data=speech["audio"])

generated_topic = image2text("https://img.freepik.com/premium-photo/dark-mansion-with-lights-forest_839169-21855.jpg")

historia = crear_historia(generated_topic)
crear_audio(historia)